# **RetinaNet**

Postup vytvorenia modelu RetinaNet v tomto notebooku je založený na <a href="https://github.com/benihime91/pytorch_retinanet/blob/master/demo.ipynb">originálnom notebooku.</a>

### Príprava prostredia

In [ ]:
!pip install -Uqq pytorch-lightning==1.0.0 omegaconf
!pip install -Uqq git+https://github.com/albumentations-team/albumentations
!pip uninstall torchtext
!pip install pycocotools
!git clone https://github.com/benihime91/pytorch_retinanet.git

In [ ]:
import warnings
import os
import sys

warnings.filterwarnings('ignore')
sys.path.append("pytorch_retinanet/")
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%cd pytorch_retinanet

### Príprava dát

In [ ]:
# Cesty k obrázkom
TRAIN_IMAGE_PATH = "../../dataset/train/XML"   # trénovacia množina
VALID_IMAGE_PATH = "../../dataset/val/XML_"    # validačná množina
TEST_IMAGE_PATH  = "../../dataset/TP_test/XML" # testovacia množinaň
# Cesty k anotáciám 
TRAIN_ANNOT_PATH = "../../dataset/train/XML"   # trénovacia množina
VALID_ANNOT_PATH = "../../dataset/val/XML_"    # validačná množina
TEST_ANNOT_PATH  = "../../dataset/TP_test/XML" # testovacia množina

In [ ]:
import pandas as pd
from PIL import Image
import cv2
import numpy as np

from utils.pascal import convert_annotations_to_df

pd.set_option("display.max_colwidth", None)
np.random.seed(123)

**Vytvorenie CSV súborov z XML anotácií**

In [ ]:
train_df = convert_annotations_to_df(TRAIN_ANNOT_PATH, TRAIN_IMAGE_PATH, image_set="train")
valid_df = convert_annotations_to_df(VALID_ANNOT_PATH, VALID_IMAGE_PATH, image_set="test")
test_df  = convert_annotations_to_df(TEST_ANNOT_PATH, TEST_IMAGE_PATH, image_set="test")

def remove_invalid_annots(df):
    """
    Removes annotations where xmax, ymax < xmin,ymin
    from the given dataframe
    """
    df = df[df.xmax > df.xmin]
    df = df[df.ymax > df.ymin]
    df.reset_index(inplace=True, drop=True)
    return df

# Odstránenie invalidných anotácií
train_df = remove_invalid_annots(train_df)
valid_df = remove_invalid_annots(valid_df)
test_df  = remove_invalid_annots(test_df)

In [ ]:
# Cesta pre uloženie CSV súborov
TRAIN_CSV = "../train_data.csv"
VALID_CSV = "../valid_data.csv"
TEST_CSV  = "../test_data.csv"

# Načítanie CSV súborov
train_df.to_csv(TRAIN_CSV, index=False)
valid_df.to_csv(VALID_CSV, index=False)
test_df.to_csv(TEST_CSV, index=False)

train_df = pd.read_csv(TRAIN_CSV)
valid_df = pd.read_csv(VALID_CSV)
test_df  = pd.read_csv(TEST_CSV)

**Vytvorenie Label Map pre použitie pri vizuálizácií**

In [ ]:
from utils.pascal import generate_pascal_category_names

LABEL_MAP = generate_pascal_category_names(train_df)
LABEL_MAP

**Nastavenie konfigurácií hyperparametrov pre trénovanie**

In [ ]:
# Nastavenie počtu epoch pre trénvanie
NUM_TRAIN_EPOCHS = 50

# Nastavenie hyperparametrov
from omegaconf import OmegaConf

# Načítanie hparams.ymal súboru použitím Omegaconf
hparams = OmegaConf.load("hparams.yaml")

hparams.dataset.kind        = "csv"
hparams.dataset.trn_paths   = TRAIN_CSV
hparams.dataset.valid_paths = VALID_CSV
hparams.dataset.test_paths  = TEST_CSV

hparams.dataloader.train_bs = 4
hparams.dataloader.valid_bs = 16
hparams.dataloader.test_bs  = 16

hparams.model.num_classes   = len(LABEL_MAP) - 1
hparams.model.backbone_kind = "resnet101"
hparams.model.min_size      = 800
hparams.model.max_size      = 1333
hparams.model.pretrained    = True 

hparams.transforms  =  [
    {"class_name": "albumentations.HorizontalFlip", "params": {"p": 0.5} },
    {"class_name": "albumentations.RandomBrightnessContrast", "params": {"p": 0.5} },
]

hparams.optimizer = {
    "class_name": "torch.optim.SGD",
    "params"    : {"lr": 0.001, "weight_decay": 0.0005, "momentum":0.9},
    }

hparams.scheduler = {
    "class_name" : "torch.optim.lr_scheduler.CosineAnnealingLR",
    "params"     : {"T_max": NUM_TRAIN_EPOCHS},
    "monitor"    : None,
    "interval"   : "epoch",
    "frequency"  : 1
    }

print(OmegaConf.to_yaml(hparams))

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, EarlyStopping

from model import RetinaNetModel

pl.seed_everything(123)

### Trénovanie modelu

In [ ]:
lr_logger  = LearningRateMonitor(logging_interval="step")

# technika predčasného ukončenia trénovania
early_stopping = EarlyStopping(
    monitor='val_loss',   # monitorovaná metrika
    patience=15,          # počet epoch bez zlepšenia metriky, predtým ako sa zastaví trénovanie
    mode='min',           # 'min' pre minimalizáciu metriky, 'max' pre maximalizáciu
    min_delta=0.0001      # minimálna zmena, aby sa kvalifikovalo ako zlepšenie
)

trainer    = Trainer(precision=16,
                     accelerator='gpu',
                     devices=1, 
                     callbacks=[lr_logger,  early_stopping],
                     max_epochs=NUM_TRAIN_EPOCHS)

litModel = RetinaNetModel(conf=hparams)
trainer.fit(litModel)

### Testovanie modelu použitím COCO metrík

In [ ]:
trainer.test(litModel)

### Exportovanie váh natrénovaného modelu

In [ ]:
import torch

PATH = f"trained_weights.pth"
torch.save(litModel.net.state_dict(), PATH)

### Načítanie modelu

In [ ]:
import logging
import torch

from retinanet import Retinanet

logger = logging.getLogger("lightning")
PATH = "trained_weights.pth" # Cesta ku váham modelu
state_dict = torch.load(PATH)
model_args = hparams.model

MODEL = Retinanet(**model_args, logger=logger)
MODEL.load_state_dict(state_dict)
MODEL.eval()
MODEL.to("cuda:0");

### Vizualizácia predikcií na testovacích dátach

**Definovanie fukncií pre vizualizáciu**

In [ ]:
from albumentations.pytorch import ToTensorV2
from PIL import Image
from torchvision.ops import nms

import os
import numpy as np
import cv2
import albumentations as A
import matplotlib.pyplot as plt

@torch.no_grad()
def get_preds(path):
    image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

    INFER_TRANSFORMS = A.Compose([
        A.ToFloat(max_value=255.0, always_apply=True),
        ToTensorV2(always_apply=True)
        ])

    TENSOR_IMAGE = INFER_TRANSFORMS(image=image)["image"].to("cuda:0")
    PREDICTIONS  = MODEL.predict([TENSOR_IMAGE])
    return PREDICTIONS[0]

def detect(image_path, threshold=0.5, nms_threshold=0.01):
    
    preds = get_preds(image_path)

    mask = preds['scores'] > threshold
    boxes = preds['boxes'][mask]
    labels = preds['labels'][mask]
    scores = preds['scores'][mask]
 
    if len(scores) > 0:
        keep_indices = nms(boxes, scores, nms_threshold)

        boxes = boxes[keep_indices]
        labels = labels[keep_indices]
        scores = scores[keep_indices]

    return boxes.cpu().numpy(), labels.cpu().numpy(), scores.cpu().numpy()

def draw_on_image(image_path, boxes, scores, classes, label_map=LABEL_MAP):
    image = Image.open(image_path)
    image = np.array(image) / 255.
    image = viz_bbs(image, boxes, scores=scores, classes=classes, label_map=LABEL_MAP)
    return image

**Vizualizácia predikcií**

Vykonanie vizualizácie na všetky obrázky v testovacej množine a následné uloženie do súborov:

*inferenced*:
>TP - súbor pre obrázky s detegovanými objektmi
 
>TN - súbor pre obrázky bez detegovaných objektov

In [ ]:
input_folder = TEST_IMAGE_PATH

CONFIDENCE_THRESHOLD = 0.6 # minimálna pravdepodobnosť, s akou algoritmus správne deteguje objekt

# Definícia výstupného súburu, kde sa uložia otestované obrázky
output_folder = "inferenced"

tp_folder = os.path.join(output_folder, "TP") # súbor tp_folder pre obrázky s detegovanými objektmi
tn_folder = os.path.join(output_folder, "TN") # súbor tn_folder pre obrázky bez detegovaných objektov

# Vytvorenie súborov
os.makedirs(output_folder, exist_ok=True)
os.makedirs(tp_folder, exist_ok=True)
os.makedirs(tn_folder, exist_ok=True)


for image_name in os.listdir(input_folder):
    if image_name.lower().endswith('.jpg'):
       
        image_path = os.path.join(input_folder, image_name)    
        boxes, labels, scores = detect(image_path, threshold=CONFIDENCE_THRESHOLD)
        original_image = Image.open(image_path)
    
        if len(boxes) > 0:
            annotated_image = draw_on_image(image_path, boxes, scores, labels)
            
            output_path = os.path.join(tp_folder, image_name)
            annotated_image.save(output_path)
            print(f"Saved {output_path}")
        else:
            output_path = os.path.join(tn_folder, image_name)
            original_image.save(output_path)
            print(f"Saved {output_path}")
    else:
        print(f"Ignored {image_name}: not a .jpg file")
